# Clone transformers repo and checkout version 2.7.0

In [1]:
!rm -rf transformers
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 23175, done.
remote: Total 23175 (delta 0), reused 0 (delta 0), pack-reused 23175
Receiving objects: 100% (23175/23175), 13.39 MiB | 27.81 MiB/s, done.
Resolving deltas: 100% (16578/16578), done.


In [2]:
cd transformers/

/content/transformers


In [3]:
!git checkout v2.7.0

Note: checking out 'v2.7.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6f5a12a5 Release: v2.7.0


# Install packages required for running `run_glue.py`

In [4]:
!pip install -r ./examples/requirements.txt

In [5]:
!pip install boto3 filelock requests tqdm sentencepiece sacremoses tokenizers

# Download GLUE data

In [0]:
GLUE_DIR="/content/data"

In [7]:
!echo $GLUE_DIR

/content/data


In [8]:
pwd

'/content/transformers'

In [9]:
!python ./utils/download_glue_data.py --help

usage: download_glue_data.py [-h] [--data_dir DATA_DIR] [--tasks TASKS]
                             [--path_to_mrpc PATH_TO_MRPC]

optional arguments:
  -h, --help            show this help message and exit
  --data_dir DATA_DIR   directory to save data to
  --tasks TASKS         tasks to download data for as a comma separated string
  --path_to_mrpc PATH_TO_MRPC
                        path to directory containing extracted MRPC data,
                        msr_paraphrase_train.txt and msr_paraphrase_text.txt


In [10]:
!python ./utils/download_glue_data.py --data_dir $GLUE_DIR --tasks RTE

	Completed!


# Compute RoBERTa score on RTE task

In [0]:
TASK_NAME="RTE"

In [12]:
!echo $TASK_NAME

RTE


In [13]:
!echo $PYTHONPATH

/env/python


In [14]:
%set_env PYTHONPATH=/content/transformers/src:/env/python

env: PYTHONPATH=/content/transformers/src:/env/python


In [15]:
!echo $PYTHONPATH

/content/transformers/src:/env/python


In [16]:
!python ./examples/run_glue.py --help

2020-03-30 20:17:05.134152: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
usage: run_glue.py [-h] --data_dir DATA_DIR --model_type MODEL_TYPE
                   --model_name_or_path MODEL_NAME_OR_PATH --task_name
                   TASK_NAME --output_dir OUTPUT_DIR
                   [--config_name CONFIG_NAME]
                   [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                   [--max_seq_length MAX_SEQ_LENGTH] [--do_train] [--do_eval]
                   [--evaluate_during_training] [--do_lower_case]
                   [--per_gpu_train_batch_size PER_GPU_TRAIN_BATCH_SIZE]
                   [--per_gpu_eval_batch_size PER_GPU_EVAL_BATCH_SIZE]
                   [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                   [--learning_rate LEARNING_RATE]
                   [--weight_decay WEIGHT_DECAY] [--adam_epsilon ADAM_EPSILON]
                   [--max_grad_norm MAX_G

In [17]:
!python ./examples/run_glue.py \
    --model_type roberta \
    --model_name_or_path roberta-base \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 3 \
    --output_dir ../${TASK_NAME}_run \
    --overwrite_output_dir

2020-03-30 20:17:18.031933: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
03/30/2020 20:17:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/30/2020 20:17:20 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
03/30/2020 20:17:20 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": "rte",
  "hidden_